In [ ]:
using Revise
import TensorCrossInterpolation as TCI
import TCIAlgorithms: MatrixProduct
import TCIAlgorithms as TCIA
using ITensors
using Quantics

In [ ]:
N = 4
D = 2
bonddims_a = [1; fill(D, N-1); 1]
bonddims_b = [1; fill(D, N-1); 1]
@assert length(bonddims_a) == N + 1
@assert length(bonddims_b) == N + 1

localdims1 = fill(2, N)
localdims2 = fill(2, N)
localdims3 = fill(2, N)

a = TCI.TensorTrain([
    rand(bonddims_a[n], localdims1[n], localdims2[n], bonddims_a[n+1]) for n = 1:N
])
b = TCI.TensorTrain([
    rand(bonddims_b[n], localdims2[n], localdims3[n], bonddims_b[n+1]) for n = 1:N
])

ab = TCIA.MatrixProduct(a, b)
;


In [ ]:
ab

In [ ]:
tolerance = 1e-12
localdims = localdims1 .* localdims3
tci, ranks, errors =
    TCI.crossinterpolate2(Float64, ab, localdims, [ones(Int, N)]; tolerance = tolerance, loginterval=1)

In [ ]:
sitesa = [Index(d, "a=$i") for (i, d) in enumerate(localdims1)]
sitesb = [Index(d, "b=$i") for (i, d) in enumerate(localdims3)]
M = Quantics.TCItoMPS(tci)
M = Quantics.unfuse_siteinds_nosplit(M, siteinds(M), [collect(i) for i in zip(sitesa, sitesb)])

In [ ]:
ab_ref = contract(ab.a_MPO, ab.b_MPO; alg="naive")
for n in eachindex(ab_ref)
    ab_ref[n] = replaceind(ab_ref[n], ab.sites1[n], sitesa[n])
    ab_ref[n] = replaceind(ab_ref[n], ab.sites3[n], sitesb[n])
end
ab_ref[1] *= onehot(ab.links_a[1] => 1)
ab_ref[1] *= onehot(ab.links_b[1] => 1)
ab_ref[end] *= onehot(ab.links_a[end] => 1)
ab_ref[end] *= onehot(ab.links_b[end] => 1)
ab_ref

In [ ]:
ab_ref ≈ M